In [154]:
import numpy as np
import math
import qinfer
from collections import deque

## Define the gate set

### Helper functions for making common gates


In [ ]:
def S_gate(dim):
    if dim == 2:
        return np.matrix(np.diag([1,1j]))
    omega = np.exp(1j*2*np.pi/dim)
    S_gate = np.zeros((dim,dim),dtype=complex)
    for i in range(dim):
        S_gate[i,i] = omega**(i*(i+1)/2)
    return S_gate

def H_gate(dim):
    if dim == 2:
        return np.matrix(1/np.sqrt(2)*[1,1],[1,-1])
    omega = np.exp(1j*2*np.pi/dim)
    H = np.zeros((dim,dim),dtype=complex)
    for i in range(dim):
        for j in range(dim):
            H[i,j] = 1/math.sqrt(dim) * omega**(j*i)
    return H

In [ ]:
H_gate(2)

In [3]:
S = [S_gate(3),H_gate(3)]

## Finding Adjoint representation in SO(d^2-1)

Generators of $\mathfrak{s u}(d)$ are found in mathematica via the approach found in https://mathematica.stackexchange.com/questions/159014/calculate-representations-of-sun-generators



In [68]:
dim = 3
B = qinfer.tomography.gell_mann_basis(dim)
G = []
for i in range(1,len(B)):
    G.append(np.matrix(B[i].full()))
G

[matrix([[ 0.70710678+0.j,  0.        +0.j,  0.        +0.j],
         [ 0.        +0.j, -0.70710678+0.j,  0.        +0.j],
         [ 0.        +0.j,  0.        +0.j,  0.        +0.j]]),
 matrix([[ 0.40824829+0.j,  0.        +0.j,  0.        +0.j],
         [ 0.        +0.j,  0.40824829+0.j,  0.        +0.j],
         [ 0.        +0.j,  0.        +0.j, -0.81649658+0.j]]),
 matrix([[0.        +0.j, 0.70710678+0.j, 0.        +0.j],
         [0.70710678+0.j, 0.        +0.j, 0.        +0.j],
         [0.        +0.j, 0.        +0.j, 0.        +0.j]]),
 matrix([[0.        +0.j, 0.        +0.j, 0.70710678+0.j],
         [0.        +0.j, 0.        +0.j, 0.        +0.j],
         [0.70710678+0.j, 0.        +0.j, 0.        +0.j]]),
 matrix([[0.        +0.j, 0.        +0.j, 0.        +0.j],
         [0.        +0.j, 0.        +0.j, 0.70710678+0.j],
         [0.        +0.j, 0.70710678+0.j, 0.        +0.j]]),
 matrix([[0.+0.j        , 0.-0.70710678j, 0.+0.j        ],
         [0.+0.70710678j, 0.

## Determine if the center of the subgroup is the trivial center

In [122]:
# generate the adjoint of an operator in SU(d) in SO(d^2-1)
def Ad(U,G):
    d = len(G)
    Ad_U = np.matrix(np.zeros((d,d),dtype=complex))
    for i in range(d):
        for j  in range(d):
            Ad_U[i,j] = -1/2 * np.trace(G[i]*U*G[j]*np.linalg.inv(U))
    return Ad_U

def check_Center(S,G):
    dim = len(G)
    I = np.matrix(np.eye(dim))
    Ms = np.matrix(np.zeros((dim**2*len(S),dim**2),dtype=complex))
    for i,gate in enumerate(S):
        Ms[i*dim**2:(i+1)*dim**2,:] = np.kron(I,Ad(gate,G)) - np.kron(Ad(np.conj(gate.T),G),I)
        
    dim_ker = len(G)**2 - np.linalg.matrix_rank(Ms)
    return dim_ker == 1

In [ ]:
check_Center(S,G)

## Determine if the Subgroup is infinite or finite

In [ ]:
def ball_check(gate,N):
    dim = gate.shape[0]
    alphas = [np.exp(1j*2*np.pi/dim*m) for m in range(6)]
    nth_gate = gate
    for n in range(N):
        trace = np.trace(nth_gate)
        for alpha in alphas:
            if (2*(dim) - alpha*np.conj(trace) - np.conj(alpha)*trace <= 1/2): # check if it is in B
                if not np.allclose(nth_gate/nth_gate[0,0],np.eye(dim)): #check if it is in the center
                    print(nth_gate)
                    return True
        nth_gate = nth_gate @ gate # increase the power by one
    return False


def ball_check_eigs(gate,N):
    dim = gate.shape[0]
    eig,_ = np.linalg.eig(gate) 
    for n in range(N):
        for m in range(dim):
            theta = 2*np.pi*m/dim
            eig_sum = 0
            for i in range(dim):
                phi = n*np.angle(eig[i])
                eig_sum += np.sin((phi-theta)/2)**2
            if eig_sum < 1/8:
                if not np.allclose(eig,eig[0]*np.ones(dim)): # make sure all the eigenvalues arent the same
                    return True
    return False


In [175]:
def test_close(A,B):
    dim = B.shape[0]
    return math.isclose(np.abs(np.trace(np.conj(A.T)@B)),dim,rel_tol=1e-2)


def add_unique_bool(new_elems, group_elems):
    added = False
    for new_elem in new_elems:
        flag = False
        for group_elem in group_elems:
            if test_close(new_elem,group_elem):
                flag = True
                break
        if not(flag): 
            group_elems.append(new_elem)
            added = True
    return added

def add_unique_num(new_elems, group_elems):
    added = 0
    for new_elem in new_elems:
        flag = False
        for group_elem in group_elems:
            if test_close(new_elem,group_elem):
                flag = True
                break
        if not(flag): 
            group_elems.append(new_elem)
            added += 1
    return added

In [176]:
def check_Finite(S,N,lmax):
    dim = S[0].shape[0]
    G_s = [np.matrix(np.eye(dim))]
    for l in range(lmax):# check words up to length 10 starting at l = 0
        new_gates = []
        for gate in G_s: 
            if ball_check(gate,N):
                print('Infinite',gate) # if it is infinite also output the gate that is part of the ball and not the center
                return [False,-1]
            for U in S:
                new_gates.append(gate@U) # might be adding duplicate elements 
        print('l = ',l)
        if add_unique_bool(new_gates,G_s) == 0:
            return [True,len(G_s)]
    return [True,len(G_s)]


def check_Finite_eigs(S,N,lmax):
    dim = S[0].shape[0]
    G_s = [np.matrix(np.eye(dim))]
    new_index = 0
    next_index = 1
    for l in range(lmax):# check words up to length 10 starting at l = 0
        new_gates = deque()
        for gate in G_s[new_index:]: 
            if ball_check_eigs(gate,N):
                print('Infinite',gate) # if it is infinite also output the gate that is part of the ball and not the center
                return [False,-1]
            for U in S:
                new_gates.append(gate@U) # might be adding duplicate elements 
      
        num_added = add_unique_num(np.array(new_gates),G_s)
        print('l = ',l, 'current size = ', len(G_s))
        new_index = next_index
        next_index += num_added 
        if num_added == 0:
            return [True,len(G_s)]
    return [True,len(G_s)]

In [168]:
check_Finite_eigs(S,10,13)

l =  0 current size =  3
l =  1 current size =  7
l =  2 current size =  14
l =  3 current size =  26
l =  4 current size =  43
l =  5 current size =  68
l =  6 current size =  103
l =  7 current size =  142
l =  8 current size =  177
l =  9 current size =  202
l =  10 current size =  214
l =  11 current size =  216
l =  12 current size =  216


[True, 216]

In [151]:
a = [1,2,3]
a.append(4,5,6)
a

TypeError: list.append() takes exactly one argument (3 given)

## Example: Finding the dimension of the Clifford group for qudits
We expect that the dimension should be $d^3(d^2-1)$

In [177]:
dim = 4
S = [S_gate(dim),H_gate(dim)]
a,number = check_Finite_eigs(S,1,20)
print('calculated = ', number ,'expected = ',dim**3*(dim**2-1))

l =  0 current size =  3
l =  1 current size =  7
l =  2 current size =  15
l =  3 current size =  29
l =  4 current size =  55
l =  5 current size =  102
l =  6 current size =  184
l =  7 current size =  325
l =  8 current size =  568
l =  9 current size =  986
l =  10 current size =  1704
l =  11 current size =  2926
l =  12 current size =  4981
l =  13 current size =  8416


KeyboardInterrupt: 

In [164]:
def check_universal(S,N=10,lmax=100):
    dim = S[0].shape[0]
    B = qinfer.tomography.gell_mann_basis(dim)
    G = []
    for i in range(1,len(B)):
        G.append(np.matrix(B[i].full()))
    center = check_Center(S,G)
    if not center:
        return False
    ball,number = check_Finite_eigs(S,N,lmax)
    return ball

array([[ 1.00000000e+00+0.00000000e+00j, -1.38777878e-16-5.55111512e-17j,
         8.32667268e-17-3.33066907e-16j],
       [-1.38777878e-16+5.55111512e-17j,  1.00000000e+00+4.29380249e-18j,
        -1.66533454e-16-1.11022302e-16j],
       [ 8.32667268e-17+3.33066907e-16j, -1.66533454e-16+1.11022302e-16j,
         1.00000000e+00-6.28656316e-18j]])